# 📘 Task Explanation: Softmax CUDA Kernel Implementation and Profiling

## 🎯 Objective
The objective of this task is to implement a **Softmax CUDA kernel** and then use **GPU profiling tools** to analyze its performance characteristics.  
Softmax is a **reduction-heavy and numerically sensitive** operator, making it an ideal case study for understanding GPU parallelism, memory access patterns, and performance bottlenecks.

This task emphasizes both **correctness** and **performance analysis**.

---

## 🧠 Background: What Is Softmax?
For an input vector \( x \in \mathbb{R}^D \), Softmax is defined as:
\[
\text{Softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{D} e^{x_j}}
\]

In practice, a **numerically stable form** is used:
\[
\text{Softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum_{j=1}^{D} e^{x_j - \max(x)}}
\]

Softmax involves:
- A **max reduction**
- A **sum reduction**
- Elementwise exponentiation and normalization

These operations make Softmax both **compute-intensive** and **memory-sensitive**.

---

## 🧩 Part A — Softmax CUDA Kernel

### Task
Design and implement a CUDA kernel for Softmax where:
- Each row (or vector) is processed independently
- Reductions (max and sum) are parallelized
- Numerical stability is ensured via the **subtract-max trick**

### Key Design Considerations
- How to map rows to thread blocks or warps
- How to implement max and sum reductions efficiently
- Whether to use shared memory or warp-level primitives
- Minimizing redundant global memory accesses

---

## 🧠 Correctness Requirements
- Use a numerically stable Softmax formulation
- Match a CPU reference implementation within tolerance
- Handle edge cases (large/small values, varying vector length)

---

## 🧩 Part B — Profiling the Softmax Kernel

### Task
Profile the Softmax CUDA kernel using GPU profiling tools such as:
- **Nsight Compute** (kernel-level analysis)
- (Optional) **Nsight Systems** (application-level timeline)

### What to Analyze
- Kernel execution time
- Warp execution efficiency
- Memory throughput and cache behavior
- Warp stalls vs memory stalls

---

## 📊 Key Metrics to Inspect
- Occupancy and active warps
- Stall reasons (e.g., memory dependency, execution dependency)
- Global memory load efficiency
- Shared memory usage and bank conflicts (if used)

---

## 🔍 Key Questions to Answer
- Is the kernel **memory-bound or compute-bound**?
- Which stage dominates runtime: max reduction, sum reduction, or normalization?
- Are reductions efficiently parallelized?
- Could kernel fusion or reduced memory traffic improve performance?

---

## 🧪 Deliverables
You should produce:
1. A Softmax CUDA kernel implementation
2. A CPU reference for correctness verification
3. Profiling reports (Nsight Compute)
4. A short analysis explaining:
   - Performance bottlenecks
   - Dominant stall reasons
   - Possible optimization directions

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:
- How to implement reduction-heavy kernels on GPU
- Why numerical stability matters in GPU kernels
- How to interpret profiling metrics for real ML operators
- How Softmax kernels are optimized in practice

---

## 🚀 Relevance to ML Systems
Softmax is a core component in:
- Attention mechanisms
- Classification layers
- Transformer models

Efficient Softmax implementations are critical for:
- LLM training and inference
- Kernel fusion (e.g., FlashAttention)
- High-performance ML systems

---

## 🧠 Key Takeaway
> **Softmax combines numerical stability challenges with reduction-heavy computation, making it a perfect kernel for learning both CUDA optimization and GPU profiling.**


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed Jan 21 12:31:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

In [ ]:
%%writefile softmax_profile_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cstring>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

static inline float frand(unsigned int &s) {
  s = 1664525u * s + 1013904223u;
  return (s & 0x00FFFFFF) / float(0x01000000); // [0,1)
}

// ============================================================
// CPU reference: numerically stable softmax per row
// y[b, i] = exp(x[b, i] - max) / sum_j exp(x[b, j] - max)
// ============================================================
static void softmax_cpu(const float* x, float* y, int B, int D) {
  for (int b = 0; b < B; ++b) {
    const float* xb = x + b * D;
    float* yb = y + b * D;

    float m = -INFINITY;
    for (int i = 0; i < D; ++i) m = fmaxf(m, xb[i]);

    double sum = 0.0;
    for (int i = 0; i < D; ++i) sum += std::exp((double)(xb[i] - m));

    double inv = 1.0 / sum;
    for (int i = 0; i < D; ++i) yb[i] = (float)(std::exp((double)(xb[i] - m)) * inv);
  }
}

// ============================================================
// TODO: Warp reduction helpers using __shfl_down_sync
// ============================================================
__device__ __forceinline__ float warpReduceSum(float v) {
  // TODO
  return v;
}
__device__ __forceinline__ float warpReduceMax(float v) {
  // TODO
  return v;
}

// ============================================================
// TODO: Softmax CUDA kernel
// Inputs:
//   x: [B, D] row-major
// Outputs:
//   y: [B, D]
// Requirements:
//   - numerically stable subtract-max
//   - parallelize max + sum reductions
//   - avoid OOB and handle any D
// Notes:
//   - you can do 1 warp per row, 1 block per row, or multi-warp per row
// ============================================================
__global__ void softmax_kernel(const float* __restrict__ x,
                               float* __restrict__ y,
                               int B, int D) {
  // TODO
}

// ============================================================
// Accuracy checker
// ============================================================
static bool check_allclose(const float* a, const float* b, int n, float rtol, float atol) {
  for (int i = 0; i < n; ++i) {
    float av = a[i], bv = b[i];
    float diff = std::fabs(av - bv);
    float tol = atol + rtol * std::fabs(bv);
    if (diff > tol || std::isnan(av) || std::isnan(bv)) {
      printf("Mismatch at %d: got=%g ref=%g diff=%g tol=%g\n", i, av, bv, diff, tol);
      return false;
    }
  }
  return true;
}

// ============================================================
// Benchmark helper (CUDA events)
// ============================================================
template <typename Kernel, typename... Args>
static float bench_kernel_ms(Kernel k, dim3 grid, dim3 block,
                             int warmup, int iters, Args... args) {
  for (int i = 0; i < warmup; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  cudaEvent_t start, stop;
  CUDA_CHECK(cudaEventCreate(&start));
  CUDA_CHECK(cudaEventCreate(&stop));

  CUDA_CHECK(cudaEventRecord(start));
  for (int i = 0; i < iters; ++i) {
    k<<<grid, block>>>(args...);
  }
  CUDA_CHECK(cudaEventRecord(stop));
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaEventSynchronize(stop));

  float ms = 0.0f;
  CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
  CUDA_CHECK(cudaEventDestroy(start));
  CUDA_CHECK(cudaEventDestroy(stop));

  return ms / iters;
}

int main(int argc, char** argv) {
  // Args: ./softmax <B> <D> <iters>
  int B = (argc > 1) ? std::atoi(argv[1]) : 4096;
  int D = (argc > 2) ? std::atoi(argv[2]) : 1024;
  int iters = (argc > 3) ? std::atoi(argv[3]) : 200;

  printf("Softmax skeleton: B=%d D=%d iters=%d\n", B, D, iters);

  const int n = B * D;
  const size_t bytes = size_t(n) * sizeof(float);

  float* hx = (float*)std::malloc(bytes);
  float* hy_ref = (float*)std::malloc(bytes);
  float* hy_gpu = (float*)std::malloc(bytes);

  if (!hx || !hy_ref || !hy_gpu) {
    fprintf(stderr, "Host malloc failed.\n");
    return 1;
  }

  // Init inputs (include some large values to stress stability)
  unsigned int st = 123u;
  for (int i = 0; i < n; ++i) {
    float r = (frand(st) - 0.5f) * 2.0f;
    float scale = (i % 97 == 0) ? 20.0f : 3.0f; // occasional large magnitude
    hx[i] = r * scale;
  }

  // CPU reference
  softmax_cpu(hx, hy_ref, B, D);

  // Device alloc
  float *dx=nullptr, *dy=nullptr;
  CUDA_CHECK(cudaMalloc(&dx, bytes));
  CUDA_CHECK(cudaMalloc(&dy, bytes));
  CUDA_CHECK(cudaMemcpy(dx, hx, bytes, cudaMemcpyHostToDevice));
  CUDA_CHECK(cudaMemset(dy, 0, bytes));

  // ------------------------------------------------------------
  // TODO: choose launch config
  // Common choices:
  //  - 1 block per row: grid=(B), block=(some threads)
  //  - or 1 warp per row: grid=(ceil(B / warpsPerBlock)), block=(warpsPerBlock*32)
  // ------------------------------------------------------------
  dim3 block(0, 0, 1); // TODO
  dim3 grid(0, 0, 1);  // TODO

  // Correctness run
  softmax_kernel<<<grid, block>>>(dx, dy, B, D);
  CUDA_CHECK(cudaGetLastError());
  CUDA_CHECK(cudaDeviceSynchronize());

  CUDA_CHECK(cudaMemcpy(hy_gpu, dy, bytes, cudaMemcpyDeviceToHost));
  bool ok = check_allclose(hy_gpu, hy_ref, n, 1e-4f, 1e-5f);
  printf("Correctness: %s\n", ok ? "PASS" : "FAIL");

  // Benchmark
  int warmup = 20;
  float ms = bench_kernel_ms(softmax_kernel, grid, block, warmup, iters, dx, dy, B, D);
  printf("Benchmark: %.4f ms\n", ms);

  // Cleanup
  CUDA_CHECK(cudaFree(dx));
  CUDA_CHECK(cudaFree(dy));
  std::free(hx);
  std::free(hy_ref);
  std::free(hy_gpu);
  return ok ? 0 : 2;
}


In [ ]:
%%writefile run_ncu_softmax.sh
#!/usr/bin/env bash
set -euo pipefail

B="${1:-4096}"
D="${2:-1024}"
ITERS="${3:-200}"

nvcc -O3 -lineinfo -o softmax softmax_profile_skeleton.cu
./softmax "${B}" "${D}" "${ITERS}"

# Nsight Compute profiling
ncu --set full \
  --kernel-name "softmax_kernel" \
  --launch-skip 0 --launch-count 1 \
  -o "ncu_softmax_B${B}_D${D}" \
  ./softmax "${B}" "${D}" "${ITERS}"


In [ ]:
!chmod +x run_ncu_softmax.sh
!./run_ncu_softmax.sh 4096 1024 200